In [10]:
import sqlite3
import matplotlib.pyplot as plt



import matplotlib.dates as mdates
from collections import Counter
from datetime import datetime

In [13]:
import sqlite3
conn = sqlite3.connect('metadata.sqlite')
cursor = conn.cursor()


cursor.execute('''
    CREATE TABLE IF NOT EXISTS conversation_logs (
        timestamp TEXT PRIMARY KEY,
        conversation_id TEXT,
        prompt TEXT,
        response TEXT,
        original_book_id Text,
        predicted_book_id Text,
        response_type Text,
        solar_documents_return_count INT
    )
''')

conn.commit()
conn.close()

In [15]:
import sqlite3
conn = sqlite3.connect('metadata.sqlite')
cursor = conn.cursor()

cursor.execute('''
    CREATE TABLE IF NOT EXISTS conversation_logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        conversation_id TEXT,
        prompt TEXT,
        response TEXT,
        original_book_id Text,
        predicted_book_id Text,
        response_type Text,
        solar_documents_return_count INT
    )
''')

conn.commit()
conn.close()

In [22]:
import sqlite3
conn = sqlite3.connect('metadata.sqlite')
cursor = conn.cursor()

cursor.execute('DROP TABLE IF EXISTS conversation_logs')

conn.commit()
conn.close()

In [5]:
import sqlite3
conn = sqlite3.connect('metadata.sqlite')
cursor = conn.cursor()

cursor.execute('SELECT * FROM conversation_logs')

for row in cursor.fetchall():
    print(row)

conn.close()

In [6]:
def insert_conversation(timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count):
    conn = sqlite3.connect('metadata.sqlite')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO conversation_logs (timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', [timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count])
    conn.commit()
    conn.close()
    

In [7]:
import random
import time

sessions = 10

sleep_time = range(0,10)

# Prompts
prompts = [
    "What is the capital of France?",
    "How to bake a chocolate cake?",
    "Explain the theory of relativity"
]

# Responses
responses = [
    "The capital of France is Paris.",
    "To bake a chocolate cake, you need to mix flour, sugar, cocoa powder, and other ingredients, then bake it in an oven.",
    "The theory of relativity, proposed by Albert Einstein, describes the laws of physics in the context of moving frames of reference."
]

# Original Book IDs (with one value as None)
original_book_ids = [12345, None, 67890]  # Using None for NULL

# Predicted Book IDs
predicted_book_ids = [54321, 98765, 12321]

# Response Types
response_types = ["Information", "Instruction", "Explanation"]

# Solar Documents Return Count
solar_documents_return_counts = [5, 10, 3]


for sess_id in range(1,sessions+1):
    number_of_conversations = random.choice(range(0, 9))
    
    for conv_id in range(1, number_of_conversations+1):
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S.%f')
        conversation_id = sess_id
        prompt = random.choice(prompts)
        response = random.choice(responses)
        original_book_id = random.choice(original_book_ids)
        predicted_book_id = random.choice(predicted_book_ids)
        response_type = random.choice(response_types)
        solar_documents_return_count = random.choice(solar_documents_return_counts)
        insert_conversation(timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count)
        time.sleep(random.choice(sleep_time))
        

#### Topic Analysis

In [24]:
conn = sqlite3.connect('metadata.sqlite')
cursor = conn.cursor()
cursor.execute("""SELECT *
                FROM conversation_logs
               """)
rows = cursor.fetchall()
print(rows)
    
conn.close()

[]


In [3]:
def plot_generator():
    
    plot_data = {}
    
    
    # Generate Timeseries Plot
    conn = sqlite3.connect('metadata.sqlite')
    cursor = conn.cursor()
    cursor.execute('SELECT timestamp FROM conversation_logs')
    timestamps = [datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S.%f') for row in cursor.fetchall()]
    conversations_per_hour = Counter([timestamp.replace(minute=0, second=0, microsecond=0) for timestamp in timestamps])
    
    # Prepare data for plotting
    times = list(conversations_per_hour.keys())
    counts = list(conversations_per_hour.values())
    
    # Plotly data
    conversations_over_time = {
        'data': [{'x': times, 'y': counts, 'type': 'timeseries'}],
        'layout': {'title': 'Conversations Over Time'}
    }
    plot_data['conversations_over_time'] = conversations_over_time
    
    
    # Average Number of Conversations Per Session
    
    cursor.execute("""
               SELECT AVG(conversation_count) AS average_conversations_per_id
                FROM (
                SELECT conversation_id, COUNT(*) AS conversation_count
                FROM conversation_logs
                GROUP BY conversation_id
                ) AS subquery;
            """)
    plot_data['average_number_of_conversations_in_a_session'] = cursor.fetchall()[0][0]
    
    
    # Book Distribution
    
    cursor.execute('''
    SELECT original_book_id, COUNT(*) as count
    FROM conversation_logs
    WHERE original_book_id IS NOT NULL
    GROUP BY original_book_id
    ORDER BY count DESC;
    ''')
    
    book_ids = []
    counts = []
    
    for row in cursor.fetchall():
        book_ids.append(row[0])
        counts.append(row[1])
    
    
    book_distribution = {
        'data': [{'x': book_ids, 'y': counts, 'type': 'bar'}],
        'layout': {'title': 'Book Distribution'}
    }
    
    plot_data['book_distribution'] = book_distribution
    
    
    # Calcualte Accuracy
    cursor.execute("""
                SELECT COUNT(*) FROM conversation_logs
                WHERE original_book_id IS NOT NULL AND predicted_book_id IS NOT NULL AND original_book_id = predicted_book_id;
                """)
    
    matched = cursor.fetchall()[0][0]
    
    cursor.execute("""
                SELECT COUNT(*) FROM conversation_logs
                WHERE original_book_id IS NOT NULL AND predicted_book_id IS NOT NULL;
                """)
    total = cursor.fetchall()[0][0]
    book_classifier_accuracy = 0
    if total != 0:
        book_classifier_accuracy = matched/total
        
    
    plot_data['book_classifier_accuracy'] = book_classifier_accuracy
    
    
    # Response Type Distribution
    
    cursor.execute("""
               SELECT response_type, COUNT(*) as count
                FROM conversation_logs
                GROUP BY response_type
                ORDER BY count DESC;
               """)
    
    rows = cursor.fetchall()
    
    response_types = []
    counts = []
    
    for row in rows:
        response_types.append(row[0])
        counts.append(row[1])
    
    
    response_distribution = {
        'data': [{'x': response_types, 'y': counts, 'type': 'bar'}],
        'layout': {'title': 'Response Distribution'}
    }
    
    plot_data['response_distribution'] = response_distribution
    
    # Solar Document Book Distribution Type Distribution
    cursor.execute("""
               SELECT original_book_id, SUM(solar_documents_return_count) as Frequency
                FROM conversation_logs
                WHERE original_book_id IS NOT NULL
                GROUP BY original_book_id;
               """)
    book_ids = []
    counts = []
    
    for row in cursor.fetchall():
        book_ids.append(row[0])
        counts.append(row[1])
    
    
    solr_documents_distribution_across_books = {
        'data': [{'x': book_ids, 'y': counts, 'type': 'bar'}],
        'layout': {'title': 'Solr distribution across Books'}
    }
    
    plot_data['solr_documents_distribution_across_booksdistribution'] = solr_documents_distribution_across_books
    
    
    # Average Number of Documents Retrieved Per Session
    cursor.execute("""
               SELECT AVG(SumOfDocuments) as AvgDocumentsPerConversation
                FROM (
                    SELECT conversation_id, SUM(solar_documents_return_count) as SumOfDocuments
                    FROM conversation_logs
                    GROUP BY conversation_id
                ) as SubQuery;
               """)
    
    plot_data['average_number_of_solr_documents_fetched_in_a_session'] = cursor.fetchall()[0][0]
    
    cursor.execute("""
               SELECT COUNT(DISTINCT conversation_id) as TotalUniqueConversations
                FROM conversation_logs;
               """)
    plot_data['total_number_of_sessions'] = cursor.fetchall()[0][0]
    
    conn.close()
    
    return plot_data

In [4]:
plots = plot_generator()

In [5]:
len(plots.keys())

6

In [6]:
plots

{'conversations_over_time': {'data': [{'x': [datetime.datetime(2023, 12, 10, 0, 0)],
    'y': [45],
    'type': 'timeseries'}],
  'layout': {'title': 'Conversations Over Time'}},
 'average_number_of_conversations_in_a_session': 5.0,
 'book_distribution': {'data': [{'x': ['12345', '67890'],
    'y': [18, 10],
    'type': 'bar'}],
  'layout': {'title': 'Book Distribution'}},
 'book_classifier_accuracy': 0.0,
 'response_distribution': {'data': [{'x': ['Information',
     'Explanation',
     'Instruction'],
    'y': [18, 16, 11],
    'type': 'bar'}],
  'layout': {'title': 'Response Distribution'}},
 'solr_documents_distribution_across_booksistribution': {'data': [{'x': [3,
     5,
     10],
    'y': [17, 12, 16],
    'type': 'bar'}],
  'layout': {'title': 'Solr distribution across Books'}}}

In [ ]:
from flask import Flask, request, jsonify
import sqlite3
from datetime import datetime
from collections import Counter

app = Flask(__name__)

@app.route('/insert_conversation', methods=['POST'])
def insert_conversation_endpoint():
    data = request.json
    conn = sqlite3.connect('metadata.sqlite')
    cursor = conn.cursor()
    cursor.execute('''
        INSERT INTO conversation_logs (timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count) 
        VALUES (?, ?, ?, ?, ?, ?, ?, ?)
    ''', [data['timestamp'], data['conversation_id'], data['prompt'], data['response'], data['original_book_id'], data['predicted_book_id'], data['response_type'], data['solar_documents_return_count']])
    conn.commit()
    conn.close()
    return jsonify({"status": "success"})

@app.route('/plot_generator', methods=['GET'])
def plot_generator():
    plot_data = {}

    conn = sqlite3.connect('metadata.sqlite')
    cursor = conn.cursor()

    # Generate Timeseries Plot
    cursor.execute('SELECT timestamp FROM conversation_logs')
    timestamps = [datetime.strptime(row[0], '%Y-%m-%d %H:%M:%S.%f') for row in cursor.fetchall()]
    conversations_per_minute = Counter([timestamp.replace(second=0, microsecond=0) for timestamp in timestamps])

    # Prepare data for plotting
    times = list(conversations_per_minute.keys())
    counts = list(conversations_per_minute.values())

    # Plotly data
    conversations_over_time = {
        'data': [{'x': times, 'y': counts, 'type': 'timeseries'}],
        'layout': {'title': 'Conversations Over Time'}
    }
    plot_data['conversations_over_time'] = conversations_over_time

    # Average Number of Conversations Per Session
    cursor.execute("""
        SELECT AVG(conversation_count) AS average_conversations_per_id
        FROM (
            SELECT conversation_id, COUNT(*) AS conversation_count
            FROM conversation_logs
            GROUP BY conversation_id
        ) AS subquery;
    """)
    plot_data['average_number_of_conversations_in_a_session'] = cursor.fetchall()[0][0]

    # Book Distribution
    cursor.execute('''
        SELECT original_book_id, COUNT(*) as count
        FROM conversation_logs
        WHERE original_book_id IS NOT NULL
        GROUP BY original_book_id
        ORDER BY count DESC;
    ''')
    book_ids = []
    counts = []
    for row in cursor.fetchall():
        book_ids.append(row[0])
        counts.append(row[1])

    book_distribution = {
        'data': [{'x': book_ids, 'y': counts, 'type': 'bar'}],
        'layout': {'title': 'Book Distribution'}
    }
    plot_data['book_distribution'] = book_distribution

    # Calculate Accuracy
    cursor.execute("""
        SELECT COUNT(*) FROM conversation_logs
        WHERE original_book_id IS NOT NULL AND original_book_id = predicted_book_id;
    """)
    matched = cursor.fetchall()[0][0]

    cursor.execute("""
        SELECT COUNT(*) FROM conversation_logs
        WHERE original_book_id IS NOT NULL;
    """)
    total = cursor.fetchall()[0][0]

    book_classifier_accuracy = matched / total
    plot_data['book_classifier_accuracy'] = book_classifier_accuracy

    # Response Type Distribution
    cursor.execute("""
        SELECT response_type, COUNT(*) as count
        FROM conversation_logs
        GROUP BY response_type
        ORDER BY count DESC;
    """)
    rows = cursor.fetchall()
    response_types = []
    counts = []
    for row in rows:
        response_types.append(row[0])
        counts.append(row[1])

    response_distribution = {
        'data': [{'x': response_types, 'y': counts, 'type': 'bar'}],
        'layout': {'title': 'Response Distribution'}
    }
    plot_data['response_distribution'] = response_distribution

    conn.close()
    return jsonify(plot_data)

if __name__ == '__main__':
    app.run(debug=True)


In [23]:
import sqlite3

# Connect to the SQLite database
conn = sqlite3.connect('metadata.sqlite')

# Create a cursor object
cursor = conn.cursor()

cursor.execute('''ALTER TABLE conversation_logs RENAME TO temp_table;''')


cursor.execute('''
    CREATE TABLE IF NOT EXISTS conversation_logs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        timestamp TEXT,
        conversation_id TEXT,
        prompt TEXT,
        response TEXT,
        original_book_id Text,
        predicted_book_id Text,
        response_type Text,
        solar_documents_return_count INT
    )
''')

# Copy data from conversation_logs to conversation_logs2
cursor.execute('''
    INSERT INTO conversation_logs (timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count)
    SELECT timestamp, conversation_id, prompt, response, original_book_id, predicted_book_id, response_type, solar_documents_return_count
    FROM temp_table
''')


cursor.execute('DROP TABLE IF EXISTS temp_table')
# Commit the changes and close the connection
conn.commit()
conn.close()

"Data copied successfully from 'conversation_logs' to 'conversation_logs2'."


"Data copied successfully from 'conversation_logs' to 'conversation_logs2'."